## Select classes from kepek to output_kepek

In [10]:
import os
import shutil
import csv

# Bemeneti és kimeneti mappák
input_folder = 'kepek'
output_folder = 'output_kepek'
csv_file_path = 'kep_osztalyok.csv'

# Csak ezeket az osztályokat kezeljük
osztalyok = {'1', '2', '3'}

# Kimeneti almappák létrehozása
for osztaly in osztalyok:
    os.makedirs(os.path.join(output_folder, osztaly), exist_ok=True)

# CSV sorok ide kerülnek
csv_rows = [('class', 'path')]

# Fájlok feldolgozása
for filename in os.listdir(input_folder):
    if filename.endswith('.png'):
        try:
            # Feltételezett fájlnév formátum: nev_osztaly.png
            class_part = filename.rsplit('_', 1)[-1].split('.')[0]
            if class_part in osztalyok:
                src = os.path.join(input_folder, filename)
                dst = os.path.join(output_folder, class_part, filename)
                shutil.copy(src, dst)

                # CSV-be relatív elérési útvonalat írunk
                relative_path = os.path.join(output_folder, class_part, filename)
                csv_rows.append((class_part, relative_path))
            else:
                print(f"Kihagyva (ismeretlen osztály): {filename}")
        except Exception as e:
            print(f"Hiba: {filename} – {e}")

# CSV fájl mentése
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(csv_rows)

print(f"CSV fájl elkészült: {csv_file_path}")


CSV fájl elkészült: kep_osztalyok.csv


In [11]:
import cv2
import csv
import os

# Célméret
target_size = (224, 224)

# Center crop + resize függvény OpenCV-vel
def center_crop_and_resize_cv2(image_path, target_size):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"A kép nem olvasható: {image_path}")

    height, width = img.shape[:2]
    min_side = min(height, width)

    # Vágás koordinátái
    top = (height - min_side) // 2
    left = (width - min_side) // 2
    cropped_img = img[top:top+min_side, left:left+min_side]

    # Átméretezés
    resized_img = cv2.resize(cropped_img, target_size)

    # Mentés az eredeti helyre
    cv2.imwrite(image_path, resized_img)

# CSV beolvasás és képfeldolgozás
csv_file_path = 'kep_osztalyok.csv'

with open(csv_file_path, mode='r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # fejléc átugrása

    for row in reader:
        class_label, image_path = row
        try:
            center_crop_and_resize_cv2(image_path, target_size)
        except Exception as e:
            print(f"Hiba: {image_path} – {e}")


## HOG

In [12]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def preprocess_image(image_path):
    # 1. Kép beolvasása szürkeárnyalatosként
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # 2. CLAHE kontrasztjavítás
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_clahe = clahe.apply(img)

    # 3. HOG-deszkriptor
    hog = cv2.HOGDescriptor()
    h = hog.compute(img_clahe)

    # Opció: visszaalakíthatod képformába, ha HOG vizualizáció is kell
    return h  # ez a feature vector

# Használat
hog_vector = preprocess_image("output_kepek/1/1_1.png")
print("HOG feature shape:", hog_vector.shape)


HOG feature shape: (1031940,)


In [26]:
import cv2
import csv
import numpy as np
import os

# CSV bemenet útvonala
input_csv = 'kep_osztalyok.csv'
# HOG jellemzők kimeneti CSV
output_csv = 'hog_features.csv'

# HOG inicializálása explicit paraméterekkel
winSize = (64, 128)
blockSize = (16, 16)
blockStride = (8, 8)
cellSize = (8, 8)
nbins = 9
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)

# Kimeneti CSV fájl írása
with open(input_csv, mode='r', encoding='utf-8') as infile, open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # Fejléc létrehozása: osztály, elérési útvonal, és a HOG jellemzők
    next(reader)  # Fejléc átugrása
    feature_length = len(hog.compute(cv2.resize(np.zeros((64, 64), dtype=np.uint8), (64, 128))))  # Eltávolítottuk a felesleges zárójelet
    print(f"Várt jellemzőhossz: {feature_length}")
    header = ['class', 'image_path'] + [f'feature_{i}' for i in range(feature_length)]
    writer.writerow(header)

    for row in reader:
        label, image_path = row
        label = label.replace(',', '_').replace('\n', '')
        image_path = image_path.replace(',', '_').replace('\n', '')

        try:
            # Ellenőrizzük, hogy a kép létezik-e
            if not os.path.exists(image_path):
                print(f"Nem létező kép: {image_path}")
                continue

            # Kép beolvasása szürkeárnyalatosként
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Nem olvasható kép: {image_path}")
                continue

            # HOG-hoz ajánlott méret: 64x128
            resized_img = cv2.resize(img, (64, 128))
            if resized_img.shape != (128, 64):
                print(f"Hibás képméret: {image_path}, shape: {resized_img.shape}")
                continue

            # HOG jellemzők kinyerése
            features = hog.compute(resized_img)
            if len(features) != feature_length:
                print(f"Hibás HOG jellemzőhossz: {image_path}, várt: {feature_length}, kapott: {len(features)}")
                continue

            # A HOG jellemzők lapítása és ellenőrzés
            features_flat = features.flatten().tolist()

            # Kiírás a CSV fájlba: minden egyes jellemző külön oszlopban
            row_data = [label, image_path] + features_flat

            # Loggoljuk a kiírt adatokat, hogy ellenőrizzük a formátumot
            print(f"Kiírt sor: {row_data[:10]}... (több jellemző)")  # Csak az első 10 jellemzőt írjuk ki

            writer.writerow(row_data)
            outfile.flush()

        except Exception as e:
            print(f"Hiba a kép feldolgozásakor: {image_path} – {e}")


Várt jellemzőhossz: 3780
Kiírt sor: ['3', 'output_kepek\\3\\1000_3.png', 0.20870012044906616, 0.16293901205062866, 0.1297157257795334, 0.28479400277137756, 0.28479400277137756, 0.04540494084358215, 0.04640571400523186, 0.0582246370613575]... (több jellemző)
Kiírt sor: ['3', 'output_kepek\\3\\1001_3.png', 0.2190648466348648, 0.07928306609392166, 0.042095061391592026, 0.2991897761821747, 0.2991897761821747, 0.06420944631099701, 0.030834384262561798, 0.0560416541993618]... (több jellemző)
Kiírt sor: ['3', 'output_kepek\\3\\1002_3.png', 0.18229269981384277, 0.08253484964370728, 0.13493242859840393, 0.2949586808681488, 0.2949586808681488, 0.07679343223571777, 0.03388506919145584, 0.051661211997270584]... (több jellemző)
Kiírt sor: ['3', 'output_kepek\\3\\1003_3.png', 0.1321117877960205, 0.07947857677936554, 0.0718466117978096, 0.07422157377004623, 0.018299249932169914, 0.0, 0.0033431253395974636, 0.022937269881367683]... (több jellemző)
Kiírt sor: ['3', 'output_kepek\\3\\1004_3.png', 0.1436

## ORB

In [33]:
import cv2
import csv
import numpy as np
import os

# CSV bemenet útvonala
input_csv = 'kep_osztalyok.csv'
# ORB jellemzők kimeneti CSV
output_csv = 'orb_features.csv'

# ORB inicializálása
orb = cv2.ORB_create(nfeatures=500)  # A nfeatures paraméter a maximális detektálható kulcspontok száma

# Kimeneti CSV fájl írása
with open(input_csv, mode='r', encoding='utf-8') as infile, open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # Fejléc létrehozása: osztály, elérési útvonal, és az ORB jellemzők
    next(reader)  # Fejléc átugrása
    header = ['class', 'image_path', 'keypoints_coordinates']  # A kulcspontok koordinátái
    writer.writerow(header)

    for row in reader:
        label, image_path = row
        label = label.replace(',', '_').replace('\n', '')
        image_path = image_path.replace(',', '_').replace('\n', '')

        try:
            # Ellenőrizzük, hogy a kép létezik-e
            if not os.path.exists(image_path):
                print(f"Nem létező kép: {image_path}")
                continue

            # Kép beolvasása szürkeárnyalatosként
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Nem olvasható kép: {image_path}")
                continue

            # Kulcspontok és leírók kinyerése ORB segítségével
            keypoints, descriptors = orb.detectAndCompute(img, None)

            # Jellemzők kiírása a CSV-be
            keypoints_coordinates = [(kp.pt[0], kp.pt[1]) for kp in keypoints]  # X, Y koordináták
            keypoints_coordinates_str = [f"({x}, {y})" for x, y in keypoints_coordinates]  # Formázott string lista

            # Sor beírása a CSV-be: osztály, elérési út, kulcspontok koordinátái
            writer.writerow([label, image_path, '; '.join(keypoints_coordinates_str)])
            outfile.flush()

        except Exception as e:
            print(f"Hiba a kép feldolgozásakor: {image_path} – {e}")


## SIFT

In [ ]:
import cv2
import csv
import numpy as np
import os

# CSV bemenet és kimenet útvonalai
input_csv = 'kep_osztalyok.csv'
output_csv = 'sift_features.csv'
descriptor_dir = 'sift_descriptors'  # Könyvtár a deskriptorok mentéséhez

# SIFT inicializálása
sift = cv2.SIFT_create()

# Deskriptor könyvtár létrehozása, ha nem létezik
if not os.path.exists(descriptor_dir):
    os.makedirs(descriptor_dir)

# CSV fájl beolvasása és feldolgozása
with open(input_csv, mode='r', encoding='utf-8') as infile, open(output_csv, mode='w', newline='', encoding='utf-8') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # Fejléc
    next(reader)  # Bemeneti fejléc kihagyása
    header = ['class', 'image_path', 'num_keypoints', 'descriptor_file']
    writer.writerow(header)

    for row in reader:
        label, image_path = row
        label = label.strip()
        image_path = image_path.strip()

        try:
            # Ellenőrizzük, hogy a kép létezik-e
            if not os.path.exists(image_path):
                print(f"Nem létező kép: {image_path}")
                continue

            # Kép beolvasása szürkeárnyalatosként
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                print(f"Nem olvasható kép: {image_path}")
                continue

            # SIFT kulcspontok és deskriptorok kinyerése
            keypoints, descriptors = sift.detectAndCompute(img, None)

            # Kulcspontok számának ellenőrzése
            num_keypoints = len(keypoints) if keypoints is not None else 0
            descriptor_file = ''

            # Deskriptorok mentése külön .npy fájlba, ha vannak
            if descriptors is not None and num_keypoints > 0:
                # Fájlnév generálása: kép neve alapján, egyedi azonosítóval
                base_name = os.path.splitext(os.path.basename(image_path))[0]
                descriptor_file = os.path.join(descriptor_dir, f"{base_name}_sift.npy")
                np.save(descriptor_file, descriptors)
            else:
                print(f"Nem találhatóak kulcspontok: {image_path}")

            # Sor írása a CSV-be
            writer.writerow([label, image_path, num_keypoints, descriptor_file])

        except Exception as e:
            print(f"Hiba a kép feldolgozásakor: {image_path} – {e}")
